In [1]:
""" Ingest the Hyde geotiff data into Google Earth Engine. 
-------------------------------------------------------------------------------

Author: Rutger Hofste
Date: 20190722
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""


' Ingest the Hyde geotiff data into Google Earth Engine. \n-------------------------------------------------------------------------------\n\nAuthor: Rutger Hofste\nDate: 20190722\nKernel: python35\nDocker: rutgerhofste/gisdocker:ubuntu16.04\n\n'